In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [52]:
#Preprocecing
#First of all we will check how many null values there are in each column
df = pd.read_csv('chess_games.csv')

df_first_lines = df.head(500000)
df_first_lines.to_csv('shorterChessDf.csv', index=False)


In [54]:
# Define the function to split chess moves
def split_chess_moves(moves, turns):
    move_list = moves.split()
    separated_moves = []
    turn_number = 1
    for i in range(0, len(move_list), 3):
        if turn_number > turns:
            break
        separated_moves.append(move_list[i + 1] if i + 1 < len(move_list) else "")
        separated_moves.append(move_list[i + 2] if i + 2 < len(move_list) else "")
        turn_number += 1
    return separated_moves

In [55]:
def determine_result(result):
    if result == '1-0':
        return 1
    elif result == '0-1':
        return 0
    elif result == '1/2-1/2':
        return 2
    else:
        return -1  # for any unexpected result format

In [56]:
df = pd.read_csv('shorterChessDf.csv')  # Load the DataFrame

df['Result'] = df['Result'].apply(lambda x: determine_result(x))  # Apply the determine_result function

number_of_turns = 20  # Specify the number of turns we want to extract

# Create new columns for the specified number of turns
columns = [f"Turn {i + 1} White" for i in range(number_of_turns)] + [f"Turn {i + 1} Black" for i in range(number_of_turns)]

# Apply the split_chess_moves function to each row in the DataFrame
moves_split = df['AN'].apply(lambda x: pd.Series(split_chess_moves(x, number_of_turns)))

df[columns] = moves_split  # Combine the new columns with the original DataFrame

# Drop specified columns
df.drop(columns=['Event', 'White', 'Black', 'UTCDate', 'UTCTime', 'WhiteElo', 'BlackElo', 'WhiteRatingDiff',
                 'BlackRatingDiff', 'ECO', 'Opening', 'TimeControl', 'Termination', 'AN'], inplace=True)

# Save the modified DataFrame to a CSV file
output_file = 'modified_dataframe.csv'
df.to_csv(output_file, index=False)

df.head()

,Result,Turn 1 White,Turn 2 White,Turn 3 White,Turn 4 White,Turn 5 White,Turn 6 White,Turn 7 White,Turn 8 White,Turn 9 White,...,Turn 11 Black,Turn 12 Black,Turn 13 Black,Turn 14 Black,Turn 15 Black,Turn 16 Black,Turn 17 Black,Turn 18 Black,Turn 19 Black,Turn 20 Black
0,1,d4,d5,c4,c6,e3,a6,Nf3,e5,cxd5,...,Rac1,Qd6,Qc2,Qe6,Nb1,Bd6,a3,Nb6,Qc6,Nfd5
1,0,e4,e5,b3,Nf6,Bb2,Nc6,Nf3,d6,d3,...,Rxc7,Nxa2,Ra1,Nb4,Raxa7,Rxa7,Rxa7,Nxe4,Nxe4,Rxe4
2,1,e4,d5,exd5,Qxd5,Nf3,Bg4,Be2,Nf6,Nc3,...,Qxb7,Bg4,Qc6+,Ke7,Rae1,Rxh4,Nd5+,Kf8,Nxf6,Qxf6
3,1,e3,Nf6,Bc4,d6,e4,e6,Nf3,Nxe4,Nd4,...,Nxh3,g5+,Nxg5,fxg5+,Kxg5,Rh5+,Qxh5+,Kd7,Qf7+,Kd6
4,0,e4,c5,Nf3,d6,d4,cxd4,Nxd4,Nf6,Nc3,...,Ne2,Qd8,Bb6,Qe8,Be3,Ra6,c4,Nh5,Ng3,Nxg3


after splitting the games to  moves, we want to remove all null values because there are 2 options:
1. the game is shorter that the number of turns, and we do not want to consider it (null moves for not existing moves)
2. there are missing moves during the game. i.e. not complete data - we want to remove it.

In [95]:
dfFiltered = df.copy()
dfFiltered = dfFiltered.dropna()

rows_with_nulls = dfFiltered[dfFiltered.isnull().any(axis=1)]
rows_with_nulls

,Result,Turn 1 White,Turn 2 White,Turn 3 White,Turn 4 White,Turn 5 White,Turn 6 White,Turn 7 White,Turn 8 White,Turn 9 White,...,Turn 11 Black,Turn 12 Black,Turn 13 Black,Turn 14 Black,Turn 15 Black,Turn 16 Black,Turn 17 Black,Turn 18 Black,Turn 19 Black,Turn 20 Black


In [100]:
dfFilteredAndCompacted = dfFiltered.head(50000)
dfFilteredAndCompacted.to_csv('dfFilteredCompacted.csv', index=False)

,Result,Turn 1 White,Turn 2 White,Turn 3 White,Turn 4 White,Turn 5 White,Turn 6 White,Turn 7 White,Turn 8 White,Turn 9 White,...,Turn 11 Black,Turn 12 Black,Turn 13 Black,Turn 14 Black,Turn 15 Black,Turn 16 Black,Turn 17 Black,Turn 18 Black,Turn 19 Black,Turn 20 Black


In [97]:
dfFilteredAndCompacted = pd.read_csv('dfFilteredCompacted.csv')
dfFilteredAndCompacted

,Result,Turn 1 White,Turn 2 White,Turn 3 White,Turn 4 White,Turn 5 White,Turn 6 White,Turn 7 White,Turn 8 White,Turn 9 White,...,Turn 11 Black,Turn 12 Black,Turn 13 Black,Turn 14 Black,Turn 15 Black,Turn 16 Black,Turn 17 Black,Turn 18 Black,Turn 19 Black,Turn 20 Black
0,1,d4,d5,c4,c6,e3,a6,Nf3,e5,cxd5,...,Rac1,Qd6,Qc2,Qe6,Nb1,Bd6,a3,Nb6,Qc6,Nfd5
1,0,e4,e5,b3,Nf6,Bb2,Nc6,Nf3,d6,d3,...,Rxc7,Nxa2,Ra1,Nb4,Raxa7,Rxa7,Rxa7,Nxe4,Nxe4,Rxe4
2,1,e4,d5,exd5,Qxd5,Nf3,Bg4,Be2,Nf6,Nc3,...,Qxb7,Bg4,Qc6+,Ke7,Rae1,Rxh4,Nd5+,Kf8,Nxf6,Qxf6
3,1,e3,Nf6,Bc4,d6,e4,e6,Nf3,Nxe4,Nd4,...,Nxh3,g5+,Nxg5,fxg5+,Kxg5,Rh5+,Qxh5+,Kd7,Qf7+,Kd6
4,0,e4,c5,Nf3,d6,d4,cxd4,Nxd4,Nf6,Nc3,...,Ne2,Qd8,Bb6,Qe8,Be3,Ra6,c4,Nh5,Ng3,Nxg3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1,Nc3,g6,e4,Bg7,Bc4,c6,a3,e6,d3,...,c3,Kh8,Rc1,Re8,Bb1,Nh6,Qd2,Nf7,Ng3,Nb6
49996,0,e4,{,0.25],},c5,{,0.33],},Nf3,...,0.31],},Nc3,{,0.2],},a6,{,0.27],}
49997,0,e4,d6,d4,Nf6,f3,c5,c3,cxd4,cxd4,...,Qd6,Nc6,Nd2,Rad8,Qa3,Rxd2,h3,Qf2#,NaN,NaN
49998,1,e4,{,0.28],},e5,{,0.13],},Nf3,...,0.56],},Nxd6+,{,0.64],},Qxd6,{,0.63],}


In [98]:
rows_with_nulls = dfFilteredAndCompacted[dfFilteredAndCompacted.isnull().any(axis=1)]
rows_with_nulls

,Result,Turn 1 White,Turn 2 White,Turn 3 White,Turn 4 White,Turn 5 White,Turn 6 White,Turn 7 White,Turn 8 White,Turn 9 White,...,Turn 11 Black,Turn 12 Black,Turn 13 Black,Turn 14 Black,Turn 15 Black,Turn 16 Black,Turn 17 Black,Turn 18 Black,Turn 19 Black,Turn 20 Black
207,0,e4,e5,Nf3,Nc6,Bc4,Nf6,d3,d6,Ng5,...,Qxf1,Qg6,Bh4,Ne3,Qf2,Ng4,Qg3,Ne2+,NaN,NaN
548,0,e4,e5,d4,Qh4,Nc3,Bb4,Bd2,Bxc3,Bxc3,...,O-O-O,Bxh3,Qxh7,Qxf2,Qd3,Nb4,Qxh3,Qxc2#,NaN,NaN
586,0,e3,e5,Qh5,Qe7,Qf3,e4,Qg4,Nf6,Qe2,...,d4,Nxe3,Bxe3,Qxe3,Nd2,Nxg3,Rh3,Qxe2#,NaN,NaN
590,1,e4,d5,e5,d4,Qf3,f6,Bc4,e6,d3,...,Qxe5+,Be7,Rxe2,Nd7,Qxh5,Nf6,Qc5,Nd5,NaN,NaN
619,0,d4,g6,e4,Bg7,Nf3,Nf6,Bc4,O-O,Ng5,...,Kh1,Nxd5,Be3,Nxe3,Qd8+,Bf8,Re1,Nc6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49440,0,Nf3,Nf6,Nc3,e6,d4,Be7,e4,O-O,e5,...,Bxd6,Bh4+,Be2,Bxg3,hxg3,Qxd6,Kd1,Bf5,NaN,NaN
49514,0,e4,e5,Nc3,Nf6,Bc4,Nc6,Nf3,Nxe4,Bxf7+,...,Rxd1,Qf7,b3,Ba3+,Kb1,Rd8,Nf4,Qxf4,NaN,NaN
49551,0,e4,e5,f4,Bc5,Nf3,d6,c3,Bg4,Be2,...,Qb5,b6,Rac1,f6,Rc6,Qf7,Qa6,Bxf3,NaN,NaN
49593,0,d4,d5,c4,dxc4,e3,b5,a4,Bd7,axb5,...,cxd5,Bd7,hxg4,hxg4,Nh2,Qh4,Bxg4,Qxh2#,NaN,NaN


In [70]:
mask = dfFilteredAndCompacted.notnull().sum(axis = 1)

In [72]:
mask[mask != 41]

207      39
548      39
586      39
590      39
619      39
         ..
49440    39
49514    39
49551    39
49593    39
49997    39
Length: 402, dtype: int64

In [64]:


dfFilteredAndCompacted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 41 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Result         50000 non-null  int64 
 1   Turn 1 White   50000 non-null  object
 2   Turn 2 White   50000 non-null  object
 3   Turn 3 White   50000 non-null  object
 4   Turn 4 White   50000 non-null  object
 5   Turn 5 White   50000 non-null  object
 6   Turn 6 White   50000 non-null  object
 7   Turn 7 White   50000 non-null  object
 8   Turn 8 White   50000 non-null  object
 9   Turn 9 White   50000 non-null  object
 10  Turn 10 White  50000 non-null  object
 11  Turn 11 White  50000 non-null  object
 12  Turn 12 White  50000 non-null  object
 13  Turn 13 White  50000 non-null  object
 14  Turn 14 White  50000 non-null  object
 15  Turn 15 White  50000 non-null  object
 16  Turn 16 White  50000 non-null  object
 17  Turn 17 White  50000 non-null  object
 18  Turn 18 White  50000 non-n

In [48]:
# Counting null values in each column
null_counts = dfNew.isnull().sum()

print(null_counts)


Result              0
Turn 1 White        0
Turn 2 White        0
Turn 3 White      313
Turn 4 White      313
Turn 5 White      428
Turn 6 White      428
Turn 7 White      533
Turn 8 White      533
Turn 9 White      685
Turn 10 White     685
Turn 11 White     864
Turn 12 White     864
Turn 13 White    1068
Turn 14 White    1068
Turn 15 White    1313
Turn 16 White    1313
Turn 17 White    1610
Turn 18 White    1610
Turn 19 White    1952
Turn 20 White    1952
Turn 1 Black     2325
Turn 2 Black     2325
Turn 3 Black     2767
Turn 4 Black     2767
Turn 5 Black     3256
Turn 6 Black     3256
Turn 7 Black     3779
Turn 8 Black     3779
Turn 9 Black     4327
Turn 10 Black    4327
Turn 11 Black    4977
Turn 12 Black    4977
Turn 13 Black    5652
Turn 14 Black    5652
Turn 15 Black    6419
Turn 16 Black    6419
Turn 17 Black    7201
Turn 18 Black    7201
Turn 19 Black    7944
Turn 20 Black    7944
dtype: int64


In [ ]:

# Counting unique values in each column
unique_counts = df.nunique()

print(unique_counts)